<h1>Regressões</h1>

<p>O método de regressão é um dos mais antigos e mais utilizados em métodos analíticos quantitativos. O objetivo da regressão é produzir um modelo que melhor se ajusta a um conjunto de dados observado. Tipicamente, o modelo é uma função que descreve algum tipo de curva (linhas, parábolas etc.) e é determinado a partir de um conjunto de parâmetros. No caso da regressão linear, o modelo é a função linear e os parâmetros que o descrevem são o intercepto e inclinação.
<p>Em termos matemáticos, o objetivo da regressão é encontrar uma função de algumas variáveis ou atributos <span class="math inline">\(x_1, x_2...x_n\)</span>, que prediz o valor do rótulo <span class="math inline">\(y\)</span>. O valor predito do rótulo é denominado <span class="math inline">\(\hat{y}\)</span> e é dado por:</p>
<p><span class="math display">\[\hat{y} = f(x_1, x_2...x_n)\]</span></p>
<p>Inicialmente, a regressão <em>aprende</em> a função <span class="math inline">\(f(x_1, x_2...x_n)\)</span> a partir de dados de <strong>treinamento</strong>, ou seja, aqueles cujos valores dos rótulos y já são conhecidos. Uma vez aprendida a função (isto é, depois de determinar o intercepto e a inclinação no caso da função linear) queremos realizar predições de novos dados nunca vistos, que se encontram em outro conjunto de dados chamado conjunto de <strong>teste</strong>.</p>
<p>Neste caderno você fará um breve exercício sobre regressões lineares com uma base didática.</p>


## Regressão linear multivariada

<p>Queremos determinar os coeficientes angulares <span class="math inline">\(a_1, a_2...a_n\)</span> e o coeficiente linear (intercepto) <span class="math inline">\(a_0\)</span> do modelo de regressão linear a seguir:</p>
<p><span class="math display">\[\hat{y} = a_0 + a_1 \cdot x_1 + a_2 \cdot x_2 + ... + a_n \cdot x_n\]</span></p>

<p>Vamos usar uma base didática que reúne preços (rótulos ou y) de veículos automotivos e as suas características (atributos ou <span class="math inline">\(x_1, x_2...x_n\)</span>). </p>

Preencha as células seguintes com um ou mais comandos Python, para executar as tarefas pedidas em cada comentário. Execute as células com os comandos e mostre os resultados obtidos. Não se esqueça de importar as bibliotecas necessárias.

Você pode realizar a atividade com consulta. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Carregue o arquivo Automobile_data.csv em um DataFrame
dataset = pd.read_csv("Automobile_data.csv")
dataset.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.4,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.4,8.0,115,5500,18,22,17450


In [3]:
# Liste os nomes das colunas desse DataFrame
print(list(dataset.columns))

['symboling', 'normalized-losses', 'make', 'fuel-type', 'aspiration', 'num-of-doors', 'body-style', 'drive-wheels', 'engine-location', 'wheel-base', 'length', 'width', 'height', 'curb-weight', 'engine-type', 'num-of-cylinders', 'engine-size', 'fuel-system', 'bore', 'stroke', 'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price']


Como se vê, a base descreve diversas características de um conjunto de veículos, sendo que o último atributo (price) é o rótulo preço.

# Limpeza da base

Note que somente algumas das variáveis são numéricas. Note ainda que nossa variável alvo não é numérica, o que é um grande problema.

In [4]:
# Descubra o tipo da coluna price. Você pode mostrar o tipo da coluna inteira ou de um elemento dela.
print(dataset['price'][0].isnumeric())
print(type(dataset['price'][0]))

True
<class 'str'>


In [5]:
# Delete todas as linhas (inteiras) do DataFrame onde a variável price não é do tipo numérico.
for i, valor in enumerate(dataset['price']):
    if not valor.isnumeric():
        dataset = dataset.drop(i)

In [6]:
# Converta o atributo price para o tipo float. Para isso, utilize o método pd.to_numeric() com o parâmetro downcast="float"
dataset['price'] = pd.to_numeric(dataset['price'], downcast = 'float')

In [7]:
# Repita os dois passos anteriores para o atributo horsepower
dataset['horsepower'].replace("?", np.nan, inplace = True)
dataset.dropna(subset = ['horsepower'], inplace = True)
dataset['horsepower'] = pd.to_numeric(dataset['horsepower'], downcast = 'float')

In [8]:
# Crie uma nova coluna chamada logPrice igual ao logaritmo natural do preço utilizando a função np.log
dataset['logPrice'] = np.log(dataset['price'])
dataset.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,logPrice
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,mpfi,3.47,2.68,9.0,111.0,5000,21,27,13495.0,9.510075
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,mpfi,3.47,2.68,9.0,111.0,5000,21,27,16500.0,9.711116
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,mpfi,2.68,3.47,9.0,154.0,5000,19,26,16500.0,9.711116
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,mpfi,3.19,3.4,10.0,102.0,5500,24,30,13950.0,9.543235
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,mpfi,3.19,3.4,8.0,115.0,5500,18,22,17450.0,9.767095


Agora vamos normalizar alguns atributos.

In [9]:
# Construa um array Numpy (objeto do tipo numpy.ndarray) chamado "valores" com as colunas 'engine-size', 'horsepower', 'city-mpg', 'highway-mpg'
# do DataFrame original. Para isso, utilize o método to_numpy() do pandas
valores = dataset[['engine-size', 'horsepower', 'city-mpg', 'highway-mpg']].to_numpy()

In [10]:
# Use o Numpy para calcular a média (mean) das LINHAS do array e atribua esse array unidimensional 
# a uma variável chamada "medias". Utilize os parâmetro axis=1 e keepdims=True ao chamar o método do numpy
###keepdims será útil para a próxima questão
medias = valores.mean(axis=  1, keepdims = True)

In [11]:
# Use o Numpy para calcular o desvio-padrão (std) das LINHAS do array. 
# Utilize os parâmetro axis=1 e keepdims=True ao chamar o método do numpy
# Atribua esse array unidimensional a uma variável chamada "std".
std = valores.std(axis = 1, keepdims = True)

In [12]:
# Calcule um novo array chamado "X" a partir de (valores - medias) / std.
# Estes são os valores normalizados que formam a matriz de atributos.
x = (valores - medias) / std

Agora vamos preparar as bases de treinamento e teste.

In [13]:
# importe a classe train_test_split da biblioteca de machine learning Scikit-learn 
# Para isso, utilize o comando abaixo
from sklearn.model_selection import train_test_split

A documentação desse método está disponível em:
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [14]:
# Crie uma variável do tipo numpy.ndarray com o nome "y" a partir da coluna logPrice utilizando o método to_numpy()
y = dataset['logPrice'].to_numpy()

In [15]:
# Use a classe train_test_split da biblioteca de machine learning Scikit-learn.
# Esse passo pode ser visualizado na segunda célula da documentação do método, conforme link acima
# Gere 4 arrays a partir de X e y.
# Os arrays serão X_train, X_test, y_train e y_test, nesta ordem.
# Os arrays do conjunto de teste X_test e y_test serão 25% da amostra (test_size=0.25).
# Use random_state = 1 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 1)

In [16]:
# importe a classe LinearRegression do Scikit-learn para ajustar (fit) um modelo de regressão linear.
# utilize o comando abaixo para importar a classe:
# from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LinearRegression

A documentação desse método está disponível em:
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html?highlight=linearregression#sklearn.linear_model.LinearRegression

Use a classe LinearRegression do Scikit-learn para ajustar (fit) um modelo de regressão linear no conjunto de TREINAMENTO.

In [17]:
# Utilize a função LinearRegression().fit() para as variáveis do conjunto de treinamento (X_train e y_train).
# Use valores default em todos os parâmetros da classe.
# o novo objeto da regressão deve ser denominado "reg"
reg = LinearRegression().fit(x_train, y_train)

In [18]:
# Qual é o valor do coefiente de determinação R2 obtido? Utilize o método score
print(reg.score(x_train, y_train))

0.4543759177220664


In [19]:
# Quais são os valores do coeficientes obtidos? Utilize o atributo coef_
a = reg.coef_
print(f"Coeficientes angulares: {a}")

Coeficientes angulares: [-5.70273707e+11 -5.70273707e+11 -5.70273707e+11 -5.70273707e+11]


In [20]:
# Qual é o valor do intercepto desse modelo? Utilize o atributo intercept_
l = reg.intercept_
print(f"Coeficiente Linear: {l}")

Coeficiente Linear: 13.325704286562516


Use a classe LinearRegression do Scikit-learn para ajustar (fit) um modelo de regressão linear no conjunto de TESTE.

In [27]:
# Utilize a função LinearRegression().fit() para as variáveis do conjunto de teste (X_test e y_test).
# Use valores default em todos os parâmetros da classe.
# o novo objeto da regressão deve ser denominado "reg2"
reg2 = LinearRegression().fit(x_test, y_test)

In [35]:
# Faça a predição com o modelo ajustado (treinado) no conjunto de TESTE. Utilize o método predict
y_pred = reg.predict(x_test)
print(y_pred)

array([9.66121454, 9.52840204, 9.30971308, 9.07527704, 8.88741083,
       9.31380243, 9.07527704, 9.59767694, 9.32912226, 9.39681024,
       9.72871942, 9.01534052, 9.44002313, 9.44643183, 9.04219599,
       8.80348749, 9.55739374, 9.30971308, 9.44331903, 9.32820673,
       8.82320185, 9.68019647, 9.72664423, 9.07527704, 9.03139276,
       9.36928339, 9.23390741, 9.62318964, 9.40700311, 9.33101435,
       9.7357995 , 9.44643183, 9.32820673, 9.97047968, 9.22310419,
       9.44252558, 9.31868524, 8.94026727, 9.23213739, 8.80348749,
       9.91017694, 9.44869013, 9.32820673, 9.6364953 , 9.55739374,
       9.01534052, 9.24886102, 9.23390741, 9.21577997, 9.66451044])

In [38]:
# Qual é o valor de R2 entre o X e o y do conjunto de teste? Utilize o método score
print(reg.score(x_test, y_test))

0.3765548902361332


FIM DA ATIVIDADE. Você acabou de treinar um modelo de aprendizado de máquina (regressão linear).

O R2 indica o percentual de bom ajuste entre o modelo treinado e o conjunto de dados.